# Single Neuron Reconstruction

## Part 1: Overview of Data

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pickle

from deep_neurographs import evaluation as evaluator, inference, visualization as viz
from deep_neurographs.utils import ml_util, util


### Load Data

In [2]:
# Parameters
brain_id = "706301"
example_id = "000"
plot_bool = False

# Read graph
input_filename = f"input_graph_{brain_id}-{example_id}.pkl"
path = f"./input_graphs/{input_filename}"
with open(path, "rb") as file:
    input_graph = pickle.load(file)
print(type(input_graph))

<class 'deep_neurographs.fragments_graph.FragmentsGraph'>


### Graph Structure and Attributes

In [3]:
node = util.sample_once(input_graph.nodes)
print("Node attributes:", input_graph.nodes[node])

Node attributes: {'proposals': set(), 'radius': 2.992, 'swc_id': '8739052156', 'xyz': array([20115.559, 15391.285, 24318.135], dtype=float32)}


In [4]:
edge = util.sample_once(input_graph.edges)
print("Edge attributes:", input_graph.edges[edge])

Edge attributes: {'length': 64.49980211257935, 'radius': array([0.748, 0.748, 0.748, 0.748, 0.748, 0.748, 0.748, 0.748, 1.249,
       1.456, 1.058, 0.748, 1.   , 0.748, 0.748, 0.748, 0.748, 1.249,
       1.496, 1.673, 2.115, 2.115, 2.34 , 2.498, 1.948, 1.948, 1.8  ,
       1.948, 1.948, 1.496, 1.496, 1.673, 1.8  , 1.249, 1.   , 1.   ,
       1.058, 1.496, 1.496, 1.948, 2.115, 1.673, 1.673, 1.673, 1.673,
       2.365, 2.244, 2.365, 2.262, 1.948, 2.244, 2.244, 1.673, 1.249,
       1.   ], dtype=float16), 'xyz': array([[20338.479, 14864.654, 24037.377],
       [20337.451, 14864.604, 24037.723],
       [20336.484, 14864.55 , 24038.117],
       [20335.574, 14864.493, 24038.56 ],
       [20334.723, 14864.434, 24039.053],
       [20333.932, 14864.372, 24039.592],
       [20333.197, 14864.307, 24040.182],
       [20332.523, 14864.238, 24040.818],
       [20331.908, 14864.167, 24041.504],
       [20331.352, 14864.093, 24042.24 ],
       [20330.852, 14864.016, 24043.023],
       [20330.412, 1486

In [5]:
if plot_bool:
    viz.visualize_graph(input_graph)

### Proposal Generation

In [6]:
print("Example of a proposal:", util.sample_once(input_graph.proposals))

Example of a proposal: frozenset({835, 294})


In [7]:
print("# Proposals:", len(input_graph.proposals))

# Proposals: 450


In [8]:
if plot_bool:
    viz.visualize_proposals(input_graph, color="orange")

### Ground Truth

In [9]:
p = util.sample_once(input_graph.gt_accepts)
print("This is a proposal that should be accepted:", p)

This is a proposal that should be accepted: frozenset({213, 685})


In [10]:
groundtruth_filename = f"groundtruth_graph_{brain_id}-{example_id}.pkl"
path = f"./groundtruth_graphs/{groundtruth_filename}"
with open(path, "rb") as file:
    groundtruth_graph = pickle.load(file)

In [11]:
if plot_bool:
    viz.visualize_subset(
        input_graph,
        input_graph.gt_accepts,
        proposal_subset=True,
        color="orange",
        groundtruth_graph=groundtruth_graph
    )

## Part 2: Inference

### Features

In [12]:
features_filename = f"input_graph_features_{brain_id}-{example_id}.pkl"
with open(f"./features/{features_filename}", "rb") as file:
    features = pickle.load(file)
print("type(features):", type(features))
print("features.keys():", features.keys())

type(features): <class 'dict'>
features.keys(): dict_keys(['nodes', 'branches', 'proposals'])


In [13]:
# Node features
node = util.sample_once(input_graph.nodes)
print("Node features:", features["nodes"][node])

# Branch features
branch = util.sample_once(input_graph.edges)
print("Branch features:", features["branches"][frozenset(branch)])

# Proposal features
proposal = util.sample_once(input_graph.proposals)
print("Proposal features:", features["proposals"][proposal])

Node features: [1.         1.24880099 1.        ]
Branch features: [2.20507812 0.10569328]
Proposal features: [ 0.26029137  1.          2.45673299  2.87637806  0.58817631  0.57754725
 -0.92830652  0.65433437  0.52527463 -0.91036189  0.76496297  0.56087697
 -0.94616777  0.817469    0.62327921 -0.94112843  0.19617225  0.23444976
  0.23444976  0.37799043  0.26315789  0.26315789  0.33014354  0.31578947
  0.31578947  0.36842105  0.36842105  0.42105263  0.45454545  0.45454545
  0.44019139  0.46889952  0.34419856  0.08598255]


### Dataset

In [14]:
dataset = ml_util.init_dataset(
    input_graph,
    features,
    computation_graph=input_graph
)

In [15]:
print("Dataset Attributes:", dataset.__dict__.keys())

Dataset Attributes: dict_keys(['idxs_branches', 'idxs_proposals', 'computation_graph', 'proposals', 'node_types', 'edge_types', 'data', 'n_edge_attrs'])


In [16]:
print("Index mappings for proposals:", dataset.idxs_proposals.keys())

Index mappings for proposals: dict_keys(['idx_to_id', 'id_to_idx'])


In [17]:
proposal = util.sample_once(input_graph.proposals)
idx = dataset.idxs_proposals["id_to_idx"][proposal]
print(f"The feature vector of proposal '{proposal}' is in the {idx}-th row of the feature matrix dataset.data['proposal'].x")

The feature vector of proposal 'frozenset({104, 334})' is in the 261-th row of the feature matrix dataset.data['proposal'].x


In [18]:
print(features["proposals"][proposal] - np.array(dataset.data["proposal"].x[idx]))

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.20757942e-09  9.65816004e-10 -3.31136915e-09 -1.37973715e-09
 -1.24176344e-09  1.10378972e-09  1.51771086e-09  1.10378972e-09
 -8.27842288e-10  1.51771086e-09 -1.24176344e-09 -8.27842288e-10
  1.51771086e-09 -1.79365829e-09  0.00000000e+00 -1.37973715e-09
 -8.27842295e-10  2.17256293e-10]


### Prediction

In [20]:
# Load model
model_path = "./GraphNeuralNet-2024-10-12.pth"
model = ml_util.load_model(model_path)

# Generate proposal prediction
preds_vector = inference.predict_with_gnn(model, dataset.data, device="cpu")
preds_dict = {dataset.idxs_proposals["idx_to_id"][i]: p for i, p in enumerate(preds_vector)}

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [21]:
proposal = util.sample_once(input_graph.proposals)
print(f"Edge belief of proposal '{proposal}' is {preds_dict[proposal]}")

Edge belief of proposal 'frozenset({454, 22})' is 0.16676826775074005


### Accepted Proposals

In [22]:
acceptance_threshold = 0.75
accepts = inference.get_accepts(input_graph, preds_dict, acceptance_threshold)

print("% Proposals Accepted:", len(accepts) / input_graph.n_proposals())

% Proposals Accepted: 0.4022222222222222


### Results

In [ ]:
gt_accept_beliefs = np.array([preds_dict[p] for p in input_graph.gt_accepts])
gt_reject_beliefs = np.array([preds_dict[p] for p in input_graph.proposals if p not in input_graph.gt_accepts])

print("Median belief of proposals in gt accepts:", np.median(gt_accept_beliefs))
print("Median belief of proposals not in gt accepts:", np.median(gt_reject_beliefs))

In [ ]:
plt.figure(figsize=(18, 4.5))

plt.subplot(1, 2, 1)
plt.hist(gt_accept_beliefs, bins=30, color='tab:green', alpha=0.7, label='Proposals in gt accepts')
plt.hist(gt_reject_beliefs, bins=30, color='tab:red', alpha=0.7, label='Proposals not in gt accepts')
plt.xlabel('Belief', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Proposal Beliefs', fontsize=16)
plt.legend()

In [ ]:
# Evaluation
results = evaluator.run_evaluation(input_graph, input_graph.list_proposals(), accepts)

# Report results
results = results["Overall"]
for metric in results:
    print(f"   {metric} = {np.round(results[metric], 4)}")
